In [1]:
import tensorflow as tf
import pandas as pd
from tensorflow import feature_column as fc 
from tensorflow.keras.datasets import boston_housing

%load_ext tensorboard
# Clear any logs from previous runs
!rm -rf ./logs/ 

In [2]:
tf.__version__

'2.0.0'

In [3]:
(x_train, y_train), (x_test, y_test) = boston_housing.load_data()

In [4]:
features = ['CRIM', 'ZN', 'INDUS','CHAS','NOX','RM','AGE',
            'DIS', 'RAD', 'TAX', 'PTRATIO', 'B', 'LSTAT']
 
x_train_df = pd.DataFrame(x_train, columns= features)
x_test_df = pd.DataFrame(x_test, columns= features)
y_train_df = pd.DataFrame(y_train, columns=['MEDV'])
y_test_df = pd.DataFrame(y_test, columns=['MEDV'])
x_train_df.head()


,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT
0,1.23247,0.0,8.14,0.0,0.538,6.142,91.7,3.9769,4.0,307.0,21.0,396.90,18.72
1,0.02177,82.5,2.03,0.0,0.415,7.610,15.7,6.2700,2.0,348.0,14.7,395.38,3.11
2,4.89822,0.0,18.10,0.0,0.631,4.970,100.0,1.3325,24.0,666.0,20.2,375.52,3.26
3,0.03961,0.0,5.19,0.0,0.515,6.037,34.5,5.9853,5.0,224.0,20.2,396.90,8.01
4,3.69311,0.0,18.10,0.0,0.713,6.376,88.4,2.5671,24.0,666.0,20.2,391.43,14.65


In [5]:
feature_columns = []
for feature_name in features:
    feature_columns.append(fc.numeric_column(feature_name, dtype=tf.float32))

print(feature_columns)


[NumericColumn(key='CRIM', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='ZN', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='INDUS', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='CHAS', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='NOX', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='RM', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='AGE', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='DIS', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='RAD', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='TAX', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='PTRATIO', shape=(1

In [6]:
def estimator_input_fn(df_data, df_label, epochs=20, shuffle=True, batch_size=64):
    def input_function():
        ds = tf.data.Dataset.from_tensor_slices((dict(df_data), df_label))
        if shuffle:
            ds = ds.shuffle(100)
        ds = ds.batch(batch_size).repeat(epochs)
        return ds
    return input_function

train_input_fn = estimator_input_fn(x_train_df, y_train_df)
val_input_fn = estimator_input_fn(x_test_df, y_test_df, epochs=1, shuffle=False)

In [7]:
linear_est = tf.estimator.LinearRegressor(feature_columns=feature_columns, model_dir = 'logs/func/')
linear_est.train(train_input_fn, steps=100)
result = linear_est.evaluate(val_input_fn)
print(result)

W1218 11:09:50.886815 140064823355136 deprecation.py:506] From /home/am/anaconda3/envs/tf2p0/lib/python3.6/site-packages/tensorflow_core/python/ops/resource_variable_ops.py:1630: calling BaseResourceVariable.__init__ (from tensorflow.python.ops.resource_variable_ops) with constraint is deprecated and will be removed in a future version.
Instructions for updating:
If using Keras pass *_constraint arguments to layers.
W1218 11:09:50.887647 140064823355136 deprecation.py:323] From /home/am/anaconda3/envs/tf2p0/lib/python3.6/site-packages/tensorflow_core/python/training/training_util.py:236: Variable.initialized_value (from tensorflow.python.ops.variables) is deprecated and will be removed in a future version.
Instructions for updating:
Use Variable.read_value. Variables in 2.X are initialized automatically both in eager and graph (inside tf.defun) contexts.
W1218 11:09:50.975614 140064823355136 base_layer.py:1814] Layer linear/linear_model is casting an input tensor from dtype float64 to 

{'average_loss': 69.88763, 'label/mean': 23.078432, 'loss': 71.28754, 'prediction/mean': 23.1463, 'global_step': 100}


In [8]:
result = linear_est.predict(val_input_fn)

In [9]:
for pred,exp in zip(result, y_test[:32]):
    print("Predicted Value: ", pred['predictions'][0], "Expected: ", exp)

W1218 11:09:54.861707 140064823355136 base_layer.py:1814] Layer linear/linear_model is casting an input tensor from dtype float64 to the layer's dtype of float32, which is new behavior in TensorFlow 2.  The layer has dtype float32 because it's dtype defaults to floatx.

If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.



Predicted Value:  4.354791 Expected:  7.2
Predicted Value:  23.827093 Expected:  18.8
Predicted Value:  21.980284 Expected:  19.0
Predicted Value:  23.841606 Expected:  27.0
Predicted Value:  22.681902 Expected:  22.2
Predicted Value:  21.987122 Expected:  24.5
Predicted Value:  29.781796 Expected:  31.2
Predicted Value:  25.681396 Expected:  22.9
Predicted Value:  20.59607 Expected:  20.5
Predicted Value:  24.434635 Expected:  23.2
Predicted Value:  9.83626 Expected:  18.6
Predicted Value:  23.186514 Expected:  14.5
Predicted Value:  23.520802 Expected:  17.8
Predicted Value:  23.026697 Expected:  50.0
Predicted Value:  19.100817 Expected:  20.8
Predicted Value:  24.910473 Expected:  24.3
Predicted Value:  20.695896 Expected:  24.2
Predicted Value:  21.711885 Expected:  19.8
Predicted Value:  24.20073 Expected:  19.1
Predicted Value:  26.578724 Expected:  22.7
Predicted Value:  19.753174 Expected:  12.0
Predicted Value:  5.5827847 Expected:  10.2
Predicted Value:  21.699928 Expected: 

In [10]:
%tensorboard --logdir logs/func

Reusing TensorBoard on port 6006 (pid 6824), started 0:15:36 ago. (Use '!kill 6824' to kill it.)